In [ ]:
!pip uninstall transformers

In [ ]:
# Current stable release for CPU and GPU
!pip uninstall tensorflow

In [ ]:
!pip3 uninstall torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis") #使用情感分析
classifier(
    [
        "寶寶覺得苦，但寶寶不說",
        "我愛寶寶"
    ]
)

## 1.從  HuggingFace 下載 Dataset

In [ ]:
# 如果沒有裝 Hugging Face Datasets 的 Library 的話，可以用這個指令來裝 pip install datasets
# !pip install datasets

In [ ]:
# 使用下面的程式碼，可以先來看資料的資訊。用load_dataset_builder 不會把資料下載下來。

from datasets import load_dataset_builder
ds_builder = load_dataset_builder("poem_sentiment")
print(ds_builder.info.description)
print(ds_builder.info.features)

## 2.載入本地端的 Dataset

In [ ]:
# # 除了 sentiment = load_dataset("poem_sentiment") 來載入 Hugging Face Hub 上的 dataset。

# # 如果要載入 CSV 的 dataset 的話，可以用下面的程式碼：
# csv_dataset = load_dataset("csv", data_files="my_dataset.csv")


# # 依此類推，我們可以透過這種方式，來載入 txt、JSON 還有pickled dataframe
# txt_dataset = load_dataset("text", data_files="my_dataset.txt")
# json_dataset = load_dataset("json", data_files="my_dataset.jsonl") #注意這裡用的是 JSON Lines 的格式
# pandas_dataset = load_dataset("pandas", data_files="my_dataset.pkl")


# #還可以指定分隔符號和欄位，這裡分隔符號就要視你本身的 dataset 而定。
# csv_dataset = load_dataset("csv", data_files="my_dataset.csv",sep=",",names=["text", "label"])

## 3.載入雲端的 Dataset

In [ ]:
# # 很多時候我們自己的 dataset 放在 AWS S3 或是 Azure Blob 這類的 Object storage 裡面

# #這把整個 dataset 載下來的程式碼，如果你不在 Jupyter notebook 裡下載，要在 terminal 裡下載的話，就把驚嘆號給去掉。
# dataset_url = "https://your.dataset/url"
# !wget {dataset_url}


# #也可以直接把網址放進去，來載入遠端的 dataset。
# url = "https://your.dataset/url"
# remote_dataset = load_dataset("csv", data_files=url)


# #很多時候，你會把 training dataset 和 test dataset 分開成不同的檔案，可以用下面的程式碼一口氣合併且載入 dataset。這是一個很方便的做法。
# url = "https://your.dataset/url"
# data_files = {
#     "train": url + "train.json.gz",
#     "test": url + "json.gz",
# }

# # 這裡可以省下解壓縮 gz 檔的動作，直接 load 成 dataset，非常的方便實用
# remote_dataset = load_dataset("json", data_files=data_files)

## Hugging Face Datasets Library

In [ ]:
from datasets import load_dataset
sentiment = load_dataset("poem_sentiment")
sentiment

In [ ]:
# 接著我們可以來切資料
train_ds = sentiment["train"]
valid_ds = sentiment["validation"]
test_ds = sentiment["test"]
# 我們也可以用這種方式來切資料
dataset_train = load_dataset("rotten_tomatoes", split="train")



# 接著可以直接把 dataset 轉成 Pandas
import pandas as pd
sentiment.set_format(type="pandas")
df = sentiment["train"][:]
df.head(10)


# 我們可以使用int2str 來把 label 轉成文字。
def label_int2str(row):
    return sentiment["train"].features["label"].int2str(row)
df["label_name"] = df["label"].apply(label_int2str)
df.head(10)


# 我們也可以畫出圖片，來看看這個dataset 的 label 分佈。
import matplotlib.pyplot as plt
df["label_name"].value_counts().plot.barh()
plt.title("Poem Classes")
plt.show()


# 不要pandas 格式的話，可以這樣子改回來
sentiment.reset_format()


# 也可以把 pandas 處理過的轉成新的 dataset
from datasets import Dataset
label_name_dataset = Dataset.from_pandas(df)
label_name_dataset


# 可以這樣子 shuffle 資料
sentiment_train = sentiment["train"].shuffle(seed=5566).select(range(100))


# 也可以用 filter，來過濾資料，這裡用詩句的長度。
sentiment_filtered = sentiment.filter(lambda x: len(x["verse_text"]) > 30)
sentiment_filtered


# 當然常見的 map 也是有的，這裡我們把詩句轉成文字長度。注意這裡的batched=True，我們之後還會再見到它。
new_dataset = sentiment.map(
    lambda x: {"verse_text": [ len(o) for o in x["verse_text"] ] }, batched=True
)
new_dataset['test'][:3]

## Hugging Face Transformer Pipeline and TF model

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    ["Only those who will risk going too far can definitely find out how far one can go.",
     "Baby shark, doo doo doo doo doo doo, Baby shark!"]
)

In [ ]:
# 只要用 TFAutoModel 就可以載入想要的 transformer model 了。程式碼參考如下。
from transformers import TFAutoModel
tf_model = TFAutoModel.from_pretrained(model_name)

# 有的模型存在 PyTorch 的版本，TensorFlow 的版本是不存在的。透過 Hugging Face 能讓你用 TensorFlow 使 PyTorch 的 transformer。
# 下面的程式碼中的 xlm-roberta-base，就是只存在於 PyTorch 版本的 transformer。
tf_model = TFAutoModel.from_pretrained("xlm-roberta-base", from_pt=True)

# 在 Hugging Face 的慣例中，pt 一般就是指 PyTorch ，而 tf 是指 TensorFlow。
# 許多內建的功能，都只要再加上 TF 的前綴，就會變成 TensorFlow 版本的功能了。

In [ ]:
# Requires the latest pip
!pip install --upgrade pip

# Current stable release for CPU and GPU
!pip install tensorflow

# Or try the preview build (unstable)
!pip install tf-nightly